In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import re
import numpy as np
import pickle
from nltk.tokenize import RegexpTokenizer, word_tokenize
from nltk.stem import WordNetLemmatizer

In [2]:
user_data = pd.read_csv('data/RAW_interactions.csv')

recipe_data = pd.read_csv('data/RAW_recipes.csv')

recipes_PP = pd.read_csv('data/PP_recipes.csv')

users_PP = pd.read_csv('data/PP_users.csv')

pickled_map = pd.read_pickle('data/ingr_map.pkl')

In [3]:
# Creating a function to perform cleaning steps at once
stopwords_list = stopwords.words('english')

no_bad_chars = re.compile('[!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n - ]')
no_nums = re.compile('[\d-]')

def clean_text(text):
    #text = no_nums.sub('', text) 
    text = no_bad_chars.sub(' ', text) 
    text = text.lower() 
    text = ' '.join(word for word in text.split() if word not in stopwords_list)
    return text


In [4]:
recipe_data['description'] = recipe_data['description'].astype(str)
descr_cleaned = recipe_data['description'].apply(clean_text)

recipe_data['name'] = recipe_data['name'].astype(str)
names_cleaned = recipe_data['name'].apply(clean_text)


In [5]:
# how many recipe names include the word 'vegetarian'?
len(names_cleaned[names_cleaned.str.contains('vegetarian')])


928

In [6]:
# how many recipe descriptions include the word 'vegetarian'?
len(descr_cleaned[descr_cleaned.str.contains('vegetarian')])

3784

In [7]:
# how many recipe names include the word 'vegan'?
len(names_cleaned[names_cleaned.str.contains('vegan')])

1362

In [8]:
# how many recipe descriptions include the word 'vegan'?
len(descr_cleaned[descr_cleaned.str.contains('vegan')])

1909

In [9]:
len(recipe_data[recipe_data['ingredients'].str.contains('vegetarian')])


293

In [10]:
len(recipe_data[recipe_data['ingredients'].str.contains('vegan')])


367

In [11]:
len(recipe_data[recipe_data['ingredients'].str.contains('gluten-free')])


305

In [12]:
len(recipe_data[recipe_data['ingredients'].str.contains('gluten free')])


54

In [13]:
len(recipe_data[recipe_data['name'].str.contains('gluten-free')])


0

In [14]:
len(recipe_data[recipe_data['name'].str.contains('gluten free')])


939

In [49]:
len(recipe_data[recipe_data['description'].str.contains('gluten free')])


571

In [55]:
len(recipe_data[recipe_data['tags'].str.contains('gluten-free')])

5743

In [56]:
len(recipe_data[recipe_data['tags'].str.contains('vegan')])

10012

In [57]:
len(recipe_data[recipe_data['tags'].str.contains('vegetarian')])

35651

In [58]:
#print(recipe_data[recipe_data['name'].str.contains('gluten free')])

In [48]:
len(recipe_data[recipe_data['description'].str.contains('gluten-free')])


473

In [18]:
meat = ['ham', 'beef', 'meat', 'chicken', 'pork', 'bacon', 'sausage', 'lamb', 'veal', 'turkey', 'steak', 'rib', 'frankfurter', 'duck', 'poultry', 'goat', 'liver', 'hen', 'quail', 'brisket', 'goose']
seafood = ['fish', 'shrimp', 'seafood', 'crab', 'lobster', 'clam', 'oyster', 'scallop', 'mussel', 'cod', 'salmon', 'halibut', 'shellfish', 'roe', 'tuna', 'caviar', 'pollock', 'yellowtail', 'squid', 'calamari', 'octopus', 'crawfish', 'crayfish', 'sardine', 'trout', 'flounder', 'anchovy', 'bass', 'haddock', 'sole' ]
animal_other = ['egg', 'honey']
dairy = ['milk', 'cheese', 'yogurt', 'mayonnaise', 'butter', 'margarine', 'cream']
gluten_free = ['gluten free', 'gluten-free']


vegan = ['ham', 'beef', 'meat', 'chicken', 'pork', 'bacon', 'sausage', 'lamb', 'veal', 'turkey', 'steak', 'rib', 'frankfurter', 'duck', 'poultry', 'goat', 'liver', 'hen', 'quail', 'brisket', 'goose','fish', 'shrimp', 'seafood', 'crab', 'lobster', 'clam', 'oyster', 'scallop', 'mussel', 'cod', 'salmon', 'halibut', 'shellfish', 'roe', 'tuna', 'caviar', 'pollock', 'yellowtail', 'squid', 'calamari', 'octopus', 'crawfish', 'crayfish', 'sardine', 'trout', 'flounder', 'anchovy', 'bass', 'haddock', 'sole','egg', 'honey','milk', 'cheese', 'yogurt', 'mayonnaise', 'butter', 'margarine', 'cream']

vegetarian = ['ham', 'beef', 'meat', 'chicken', 'pork', 'bacon', 'sausage', 'lamb', 'veal', 'turkey', 'steak', 'rib', 'frankfurter', 'duck', 'poultry', 'goat', 'liver', 'hen', 'quail', 'brisket', 'goose','fish', 'shrimp', 'seafood', 'crab', 'lobster', 'clam', 'oyster', 'scallop', 'mussel', 'cod', 'salmon', 'halibut', 'shellfish', 'roe', 'tuna', 'caviar', 'pollock', 'yellowtail', 'squid', 'calamari', 'octopus', 'crawfish', 'crayfish', 'sardine', 'trout', 'flounder', 'anchovy', 'bass', 'haddock', 'sole']


In [38]:
len(recipe_data['description'].isin(gluten_free))

231637

In [90]:
len([recipe_data['ingredients'].any() in (vegan) == True])

1